In [66]:
import morph_kgc
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import datetime
from rdflib.plugin import register, Serializer, Parser

# We preprocess the data
df = pd.read_parquet('data/outsiders_2021.parquet', engine='pyarrow')

# Tipo de Contrato
tipos_contrato = {1: 'goods' , 2: 'services' , 3:'works' , 21: 'services' , 31:'works'}
df["Tipo de Contrato"] = df["Tipo de Contrato"].map(tipos_contrato)

def datetimeNone(x,y):
    try:
        z = x + ' ' + y
        datetime.datetime.strptime(z, '%Y-%m-%d %H:%M:%S')
        return z + '.000000+01:00'
    except (TypeError, ValueError):
        return '1111-11-11T11:11:11.111111+01:00'
    
df["Presentación de Solicitudes (Fecha)"] = df.apply(lambda x: datetimeNone(x["Presentación de Solicitudes (Fecha)"],["Presentación de Solicitudes (Hora)"]), axis=1)
print(df["Presentación de Solicitudes (Fecha)"].unique())


# Estado
# estado = {'ANUL': "cancelled" , 2: 'services' , 3:'works' , 21: 'services' , 31:'works'}
# df["Estado"] = df["Estado"].map(estado)
# print(df["Estado"].unique())

# Resultado
# resultado = {'ANUL': "cancelled" , 2: 'services' , 3:'works' , 21: 'services' , 31:'works'}
# df["Resultado"] = df["Estado"].map(resultado)
# print(df["Resultado"].unique())



#print(df["Tipo de Contrato"])
#print(df["Tipo de Contrato"].unique())


table = pa.Table.from_pandas(df)
pq.write_table(table, 'data/output.parquet')

TypeError: timezone() argument 1 must be datetime.timedelta, not str

In [60]:
file_path = "data/output.parquet"
#file_path = "data/2019-01-01/ocds-0c46vo-0009-DN379620-1-tender-release.xml"
rml = "rml-mappings/mappingOCDS.rml.ttl"
#rml = "rml-mappings/mapping.ttl"


# configuration file
config_morph = """
                            [DataSource1]
                            file_path=""" + file_path + """
                            mappings=""" + rml

# generate the triples and load them to an RDFlib graph
graph = morph_kgc.materialize(config_morph)
# work with the graph
graph.serialize(destination='results/parquet.nt', format='nt')

INFO | 2022-07-14 13:25:45,886 | 35 mapping rules retrieved.
INFO | 2022-07-14 13:25:45,960 | Mapping partition with 26 groups generated.
INFO | 2022-07-14 13:25:45,967 | Maximum number of rules within mapping group: 2.
INFO | 2022-07-14 13:25:45,986 | Mappings processed in 1.527 seconds.


ArrowInvalid: Casting from timestamp[us] to timestamp[ns] would result in out of bounds timestamp: -62135596800000000